In [1]:
import pandas as pd
import numpy as np
import json, os
import matplotlib.pylab as plt
from SBRP_DI import visualization, aux, fileOperations as fo
from SBRP_DI.caseStudies import postprocessing as pp

## loading data

In [2]:
# clustering
clustering = pp.loadData('hull_pm_general', res_type='clustering')
clustering_gb_max = clustering.groupby('time').max()

clustering_all = pp.loadData('hull_pm_general', res_type='clustering', path_root='../run2/times/')
clustering_all_gb_max = clustering_all.groupby('time').max()

# routing
sch17 = pp.loadData('sch17')
hull = pp.loadData('hull_pm_general')
sch17_all = pp.loadData('sch17', path_root='../run2/times/')
hull_all = pp.loadData('hull_pm_general', path_root='../run2/times/')


sch17_gb_max = sch17.groupby('time').max()
sch17_all_gb_max = sch17_all.groupby('time').max()
hull_gb_max = hull.groupby('time').max()
hull_all_gb_max = hull_all.groupby('time').max()

mask = sch17_gb_max['ObjVal'] != np.inf
mask_all = sch17_all_gb_max['ObjVal'] != np.inf
m = mask & mask_all

In [3]:
mask[~mask]

time
2021-06-07    False
2021-07-12    False
Name: ObjVal, dtype: bool

In [4]:
m[~m]

time
2021-06-07    False
2021-07-12    False
Name: ObjVal, dtype: bool

In [5]:
(
pp.count_no_solution(sch17_gb_max),
pp.count_no_solution(sch17_all_gb_max),
pp.count_no_solution(hull_gb_max),
pp.count_no_solution(hull_all_gb_max)
)

(2, 1, 0, 0)

In [6]:
S_S0_array = pp.load_S_S0(root='times/')

## results where we have an integer result with sch17 

In [11]:
table = pp.initTable(family='region C / 6-9 am', S=150, S_S0=round(S_S0_array.mean(), 2))
table.loc[0, 'picking strategy'] = 'subset'
table.loc[0, 'clust_gap']  = round(clustering_gb_max[mask]['MIPGap'].mean(), 2) * 100
table.loc[0, 'clust_time'] = round(clustering_gb_max[mask]['Runtime'].mean(), 1)
pp.rowToTable(table, [sch17_gb_max[mask], hull_gb_max[mask]],
                        ['sch17', 'hull'], row=0, latex=False)

table.loc[1, 'picking strategy'] = 'all'
table.loc[1, 'clust_gap']  = round(clustering_all_gb_max[mask_all]['MIPGap'].mean(), 2) * 100
table.loc[1, 'clust_time'] = round(clustering_all_gb_max[mask_all]['Runtime'].mean(), 1)
pp.rowToTable(table, [sch17_all_gb_max[mask_all], hull_all_gb_max[mask_all]],
                        ['sch17', 'hull'], row=1, latex=False)

family    S  S_S0 picking strategy  clust_gap  clust_time  \
0  region C / 6-9 am  150  65.6           subset        0.0       113.3   
1                NaN  NaN   NaN              all        1.0       638.2   

   sch17_gap  sch17_time  hull_gap  hull_time  hull_rel_diff  
0       0.49      2971.3       0.0       13.8          -0.05  
1       1.15      2823.7       0.0       14.9          -0.31

## all results

In [12]:
table = pp.initTable(family='region C / 6-9 am', S=150, S_S0=round(S_S0_array.mean(), 2))
table.loc[0, 'picking strategy'] = 'subset'
table.loc[0, 'clust_gap'] = round(clustering_gb_max['MIPGap'].mean(), 2) * 100
table.loc[0, 'clust_time'] = round(clustering_gb_max['Runtime'].mean(), 1)
pp.rowToTable(table, [hull_gb_max],
                     ['hull'], row=0, latex=False)

table.loc[1, 'picking strategy'] = 'all'
table.loc[1, 'clust_gap'] = round(clustering_all_gb_max['MIPGap'].mean(), 2) * 100
table.loc[1, 'clust_time'] = round(clustering_all_gb_max['Runtime'].mean(), 1)
pp.rowToTable(table, [hull_all_gb_max],
                     ['hull'], row=1, latex=False)

family    S  S_S0 picking strategy  clust_gap  clust_time  \
0  region C / 6-9 am  150  65.6           subset        0.0       109.6   
1                NaN  NaN   NaN              all        1.0       664.9   

   hull_gap  hull_time  
0       0.0       14.4  
1       0.0       15.6

## differences

In [9]:
pp.difference(sch17_gb_max[m], sch17_all_gb_max[m]).mean()

1.0038168700109609

In [10]:
diffs_hull = pp.difference(hull_gb_max, hull_all_gb_max)
diffs_hull.min(), diffs_hull.mean(), diffs_hull.max()

(-13.177265027307769, 0.24310300703104043, 13.005466928536654)